In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import numpy as np
import pandas as pd
from tensorflow.keras.datasets import mnist
from sklearn.model_selection import train_test_split

def split_mnist(n, c, d):
    # Load MNIST dataset
    (X_train, y_train), (X_test, y_test) = mnist.load_data()
    # Concatenate train and test sets
    X = np.concatenate((X_train, X_test))
    y = np.concatenate((y_train, y_test))
    
    # Create dictionary to store partitions
    partitions = {}
    for i in range(n * c):
        partitions[i] = {'X': [], 'y': []}
    
    # Iterate over classes and create partitions
    for label in range(n):
        # Get indices of samples with current label
        indices = np.where(y == label)[0]
        # Split indices into c partitions
        partitions_indices = np.array_split(indices, c)
        # Iterate over partitions and fill them with samples
        for i, partition_indices in enumerate(partitions_indices):
            # Get samples with current partition_indices
            X_partition = X[partition_indices]
            y_partition = y[partition_indices]
            # Calculate number of samples to keep from current class
            n_samples = int(d / 100 * len(partition_indices))
            # Add n_samples to partition
            partitions[label * c + i]['X'].extend(X_partition[:n_samples])
            partitions[label * c + i]['y'].extend(y_partition[:n_samples])
            # Add remaining samples to other classes partitions
            for other_label in range(n):
                if other_label != label:
                    other_indices = np.where(y == other_label)[0]
                    other_partition_indices = np.random.choice(other_indices, len(partition_indices) - n_samples, replace=False)
                    partitions[label * c + i]['X'].extend(X[other_partition_indices])
                    partitions[label * c + i]['y'].extend(y[other_partition_indices])
    return partitions
    
    # Split partitions into train and test sets
    X_train, X_test, y_train, y_test = [], [], [], []
    for i in range(n * c):
        X_partition = np.array(partitions[i]['X'])
        y_partition = np.array(partitions[i]['y'])
        X_train_part, X_test_part, y_train_part, y_test_part = train_test_split(X_partition, y_partition, test_size=0.2)
        X_train.append(X_train_part)
        X_test.append(X_test_part)
        y_train.append(y_train_part)
        y_test.append(y_test_part)
    
    # Concatenate train and test sets
    X_train = np.concatenate(X_train)
    X_test = np.concatenate(X_test)
    y_train = np.concatenate(y_train)
    y_test = np.concatenate(y_test)
    
    return (X_train, X_test, y_train, y_test)


In [4]:
# Split MNIST into 10*3 partitions where each partition contains 70% samples from one class
#X_train, X_test, y_train, y_test = split_mnist(n=10, c=9, d=90)
part = split_mnist(n=10, c=9, d=90)

In [18]:
#(X_train, y_train), (X_test, y_test) = mnist.load_data()
pd.DataFrame(part[0]['y'])[0].value_counts()

0    690
1     77
2     77
3     77
4     77
5     77
6     77
7     77
8     77
9     77
Name: 0, dtype: int64

In [ ]:
0